## Quantile dot plot 

This notebook generates quantile dot plot 

In [9]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from quantile_dotplot import ntile_dotplot
import matplotlib
import pandas as pd
import json

#### Load Article Data

In [10]:
df_product_cat = pd.read_csv('intermediate_data/Product_Article_Matching.csv')
df_product_cat = df_product_cat.reset_index().rename({"index":"id"}, axis=1)
df_product_cat.head(5)

,id,Product Category,Products,Product url,Description,Article_1_Score,Article_1_Title,Article_1_Link,Article_2_Score,Article_2_Title,Article_2_Link,Article_3_Score,Article_3_Title,Article_3_Link
0,0,Card Guide Accessories,610,https://www.digikey.com/en/products/filter/car...,Card guide accessories are hardware components...,2.680,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Wire Wrap,107,https://www.digikey.com/en/products/filter/wir...,Product Category: Wire Wrap\n\nWire wraps are ...,2.471,Multichannel impedance meter analyzes Li-Ion c...,https://www.edn.com/multichannel-impedance-met...,2.210,Robots to enable urban mining of CRMs,https://www.electronicsweekly.com/news/busines...,0.848,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...
2,2,"Electrical, Specialty Fuses",27140,https://www.digikey.com/en/products/filter/ele...,Product Category: Electrical Specialty Fuses\n...,4.734,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,3.450,Standardization and modularization in pick and...,https://www.edn.com/standardization-and-modula...,2.648,GlobalFoundries Pledges $16 Billion U.S. Inves...,https://www.eetimes.com/globalfoundries-pledge...
3,3,Circuit Breakers,73743,https://www.digikey.com/en/products/filter/cir...,Product Category: Circuit Breakers\n\nCircuit ...,1.519,A quick and practical view of USB Power Delive...,https://www.edn.com/a-quick-and-practical-view...,1.404,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,1.207,Intel’s Tech Predictions In 2009,https://www.electronicsweekly.com/blogs/manner...
4,4,"Evaluation, Development Board Enclosures",783,https://www.digikey.com/en/products/filter/eva...,"Evaluation, Development Board Enclosures are d...",4.885,Vehicle hardware security certified to ISO/SAE...,https://www.electronicsweekly.com/news/product...,3.762,Motor gate drivers enable flexible current con...,https://www.edn.com/motor-gate-drivers-enable-...,3.303,Arm Launches Zena to Accelerate AI-Defined Veh...,https://www.eetimes.com/arm-launches-zena-to-a...


#### Load Plot Data

In [11]:
df_pred = pd.read_csv("intermediate_data/05_present/Sales_Forcast_Dataframe.csv", index_col=0)
df_pred = df_pred.reset_index().rename({"index":"id"}, axis=1)
df_pred.head(5)

,id,090_day_forecast
0,0,39.0
1,1,64.0
2,2,31.0
3,3,18.0
4,4,24.0


#### Merge Names

In [12]:
df_display = df_pred.merge(df_product_cat, on="id")
df_display.head()

,id,090_day_forecast,Product Category,Products,Product url,Description,Article_1_Score,Article_1_Title,Article_1_Link,Article_2_Score,Article_2_Title,Article_2_Link,Article_3_Score,Article_3_Title,Article_3_Link
0,0,39.0,Card Guide Accessories,610,https://www.digikey.com/en/products/filter/car...,Card guide accessories are hardware components...,2.680,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,NaN,NaN,NaN,NaN,NaN,NaN
1,1,64.0,Wire Wrap,107,https://www.digikey.com/en/products/filter/wir...,Product Category: Wire Wrap\n\nWire wraps are ...,2.471,Multichannel impedance meter analyzes Li-Ion c...,https://www.edn.com/multichannel-impedance-met...,2.210,Robots to enable urban mining of CRMs,https://www.electronicsweekly.com/news/busines...,0.848,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...
2,2,31.0,"Electrical, Specialty Fuses",27140,https://www.digikey.com/en/products/filter/ele...,Product Category: Electrical Specialty Fuses\n...,4.734,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,3.450,Standardization and modularization in pick and...,https://www.edn.com/standardization-and-modula...,2.648,GlobalFoundries Pledges $16 Billion U.S. Inves...,https://www.eetimes.com/globalfoundries-pledge...
3,3,18.0,Circuit Breakers,73743,https://www.digikey.com/en/products/filter/cir...,Product Category: Circuit Breakers\n\nCircuit ...,1.519,A quick and practical view of USB Power Delive...,https://www.edn.com/a-quick-and-practical-view...,1.404,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,1.207,Intel’s Tech Predictions In 2009,https://www.electronicsweekly.com/blogs/manner...
4,4,24.0,"Evaluation, Development Board Enclosures",783,https://www.digikey.com/en/products/filter/eva...,"Evaluation, Development Board Enclosures are d...",4.885,Vehicle hardware security certified to ISO/SAE...,https://www.electronicsweekly.com/news/product...,3.762,Motor gate drivers enable flexible current con...,https://www.edn.com/motor-gate-drivers-enable-...,3.303,Arm Launches Zena to Accelerate AI-Defined Veh...,https://www.eetimes.com/arm-launches-zena-to-a...


#### Load Dot Plot Ratio

In [13]:
with open('intermediate_data/05_present/Dot_Plot_Ratio.json', 'r') as json_file:
    cat_dotplot_ratio_dict = json.load(json_file)
cat_dotplot_ratio_dict["0"][:10]

[1.10274622811379,
 1.075620195072732,
 1.065405619094332,
 1.0418655445437526,
 1.0472306270086833,
 1.073338505707987,
 1.0943292592541056,
 1.0155921947057949,
 1.0308640289342124,
 1.064380247570561]

#### Merge Datasets

#### Add in Dotplot Ratio Dict

In [14]:
df_display["dpr_dict"] = df_display["id"].apply(lambda x: cat_dotplot_ratio_dict[str(x)])
df_display.head()

,id,090_day_forecast,Product Category,Products,Product url,Description,Article_1_Score,Article_1_Title,Article_1_Link,Article_2_Score,Article_2_Title,Article_2_Link,Article_3_Score,Article_3_Title,Article_3_Link,dpr_dict
0,0,39.0,Card Guide Accessories,610,https://www.digikey.com/en/products/filter/car...,Card guide accessories are hardware components...,2.680,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,NaN,NaN,NaN,NaN,NaN,NaN,"[1.10274622811379, 1.075620195072732, 1.065405..."
1,1,64.0,Wire Wrap,107,https://www.digikey.com/en/products/filter/wir...,Product Category: Wire Wrap\n\nWire wraps are ...,2.471,Multichannel impedance meter analyzes Li-Ion c...,https://www.edn.com/multichannel-impedance-met...,2.210,Robots to enable urban mining of CRMs,https://www.electronicsweekly.com/news/busines...,0.848,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,"[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...."
2,2,31.0,"Electrical, Specialty Fuses",27140,https://www.digikey.com/en/products/filter/ele...,Product Category: Electrical Specialty Fuses\n...,4.734,Powerline module enables EV charger data links,https://www.edn.com/powerline-module-enables-e...,3.450,Standardization and modularization in pick and...,https://www.edn.com/standardization-and-modula...,2.648,GlobalFoundries Pledges $16 Billion U.S. Inves...,https://www.eetimes.com/globalfoundries-pledge...,"[1.3949346170081829, 1.3972329555288896, 1.437..."
3,3,18.0,Circuit Breakers,73743,https://www.digikey.com/en/products/filter/cir...,Product Category: Circuit Breakers\n\nCircuit ...,1.519,A quick and practical view of USB Power Delive...,https://www.edn.com/a-quick-and-practical-view...,1.404,10-octave linear-in-pitch VCO with buffered tr...,https://www.edn.com/10-octave-linear-in-pitch-...,1.207,Intel’s Tech Predictions In 2009,https://www.electronicsweekly.com/blogs/manner...,"[-43.61596367154413, -18.773336606835716, -15...."
4,4,24.0,"Evaluation, Development Board Enclosures",783,https://www.digikey.com/en/products/filter/eva...,"Evaluation, Development Board Enclosures are d...",4.885,Vehicle hardware security certified to ISO/SAE...,https://www.electronicsweekly.com/news/product...,3.762,Motor gate drivers enable flexible current con...,https://www.edn.com/motor-gate-drivers-enable-...,3.303,Arm Launches Zena to Accelerate AI-Defined Veh...,https://www.eetimes.com/arm-launches-zena-to-a...,"[9587.086862762118, -49.69653554729111, -37.85..."


In [15]:
cat_specific = "Plug Housings"

def dotplot_sku_function(df , cat):
    plt.figure(figsize=(8, 6))  #


    data = df[df["Product Category"] == cat]["dpr_dict"].iloc[0]
    prediction = df[df["Product Category"] == cat]["090_day_forecast"].iloc[0]
    dotplotdata = np.abs(prediction * np.array(data))
    date_latest = "Jun/11/2025"
    dotplt = ntile_dotplot(dotplotdata, dots=20, hist_bins="auto")
    print(type(dotplt))
    dotplt.axvline(x=prediction, color='r', linestyle='--', linewidth=2, label='Prediction')
    dotplt.set_title("90 Day Sales Prediction for \n {} \n on {} ".format(cat, date_latest),
                                                                       fontsize=20)
    #, fontweight='bold')
    dotplt.set_xlabel("\n Forecast Unit Sales Next 90 Days", fontsize=20)

    # Remove top, right, and left spines, keep only bottom
    dotplt.spines['top'].set_visible(False)
    dotplt.spines['right'].set_visible(False)
    dotplt.spines['left'].set_visible(False)
    
    # Remove y-axis ticks and labels
    dotplt.set_yticks([])
    dotplt.set_ylabel('')
    
    from matplotlib.ticker import FuncFormatter
    dotplt.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(round(x))}'))
    dotplt.tick_params(axis='x', labelsize=20) 
    #plt.tight_layout()
    dotplt.figure.savefig("figures/sfp_{}.png".format(cat.translate(str.maketrans(" /", "__"))), 
                          bbox_inches='tight', dpi=300, pad_inches=0.2)

In [16]:
dotplot_sku_function(df_display, cat_specific)

IndexError: single positional indexer is out-of-bounds

<Figure size 800x600 with 0 Axes>

#### Run for all Datasets

In [ ]:
for cat_one in df_display["Product Category"].unique():
    #print(cat_one)
    dotplot_sku_function(df_display, cat_one)
    plt.clf()

<class 'matplotlib.axes._axes.Axes'>


<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>


/var/folders/yk/12gxrg_j4_7fdrzzrf8kxwr00000gn/T/ipykernel_38598/1037950732.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 6))  #


<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<class 'matplotlib.axes._axes.Axes'>
<

<Figure size 576x432 with 0 Axes>

...

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>